In [1]:
import os
import torch
import pandas as pd
from managers.DataRepository import DataRepository
from constants import PYTORCH_MODELS_SKILLS
from dotenv import load_dotenv
load_dotenv()


True

In [1]:
import numpy as np
import torch

import os
import json
import cv2
import shutil
import pandas as pd

# from colorama import Fore, Style
from typing import List
from dotenv import load_dotenv
from managers.DataRepository import DataRepository

from ultralytics import YOLO
from ultralytics.models.yolo.detect import DetectionTrainer



In [4]:
variant = 'yolo11n.pt'
model = YOLO(variant)
args = dict(model=variant, data="jumpers.yml", epochs=3, batch=16, patience=15, lr0=0.001)
result = model.train(**args)
print("all done")


New https://pypi.org/project/ultralytics/8.3.133 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.78 🚀 Python-3.12.3 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 5898MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=jumpers.yml, epochs=3, time=None, patience=15, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train14, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, sa

train: Scanning /media/miked/Elements/Judge/FINISHED-DB-READY/ultralytics-yolo/labels/train.cache... 816 images, 1 backgrounds, 0 corrupt: 100%|██████████| 817/817 [00:00<?, ?it/s]
val: Scanning /media/miked/Elements/Judge/FINISHED-DB-READY/ultralytics-yolo/labels/val.cache... 220 images, 0 backgrounds, 0 corrupt: 100%|██████████| 220/220 [00:00<?, ?it/s]


Plotting labels to /home/miked/code/judge/runs/detect/train14/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.001' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /home/miked/code/judge/runs/detect/train14
Starting training for 3 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/3      2.32G      1.769      3.391      1.257         90        640:   2%|▏         | 1/52 [00:07<05:57,  7.01s/it]


RuntimeError: Caught RuntimeError in pin memory thread for device 0.
Original Traceback (most recent call last):
  File "/home/miked/code/judge/computervision/.venv/lib/python3.12/site-packages/torch/utils/data/_utils/pin_memory.py", line 41, in do_one_step
    data = pin_memory(data, device)
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/miked/code/judge/computervision/.venv/lib/python3.12/site-packages/torch/utils/data/_utils/pin_memory.py", line 75, in pin_memory
    {k: pin_memory(sample, device) for k, sample in data.items()}
        ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/miked/code/judge/computervision/.venv/lib/python3.12/site-packages/torch/utils/data/_utils/pin_memory.py", line 64, in pin_memory
    return data.pin_memory(device)
           ^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: CUDA error: invalid argument
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCK

In [2]:
modelparams = {
    # "balancedType" : "jump_return_push_frog_other",
    "balancedType" : "limit_10procent",
    "dim" : 224,
    "timesteps" : 16,
    "batch_size" : 1,
}
modelname = "HAR_SA_Conv3D"
modelname = "HAR_MViT"

repo = DataRepository()

STORAGE_DIR = os.getenv("STORAGE_DIR")
LABELS_FOLDER = "labels"
SUPPORTED_VIDEO_FORMATS = os.getenv("SUPPORTED_VIDEO_FORMATS")
CROPPED_VIDEOS_FOLDER = os.getenv("CROPPED_VIDEOS_FOLDER")
MODELWEIGHT_PATH = "weights"


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

connection established, loading frame paths
relative paths of framelabels loaded
Using device: cuda


In [3]:
if modelname not in PYTORCH_MODELS_SKILLS.keys():
    raise ValueError(modelname)


# TODO : update to use best val checkpoint 
modelPath = os.path.join(MODELWEIGHT_PATH, f"{modelname}.state_dict.pt")
checkpointPath = os.path.join(MODELWEIGHT_PATH, f"{modelname}.checkpoint.pt")


DIM = 224
repo = DataRepository()
model = PYTORCH_MODELS_SKILLS[modelname](modelinfo=modelparams, df_table_counts=repo.get_skill_category_counts(), skill_or_segment='skills').to(device)

if  os.path.exists(checkpointPath):
    checkpoint = torch.load(checkpointPath, weights_only=False)
    model.load_state_dict(checkpoint['model_state_dict'])
    epoch_start = checkpoint['epoch'] + 1
    losses = checkpoint['losses']
    accuracies = {} if 'accuracies' not in checkpoint.keys() else checkpoint['accuracies']
    final_classification_reports = {} if 'final_classification_reports' not in checkpoint.keys() else checkpoint['final_classification_reports']
else:
    raise ValueError(checkpointPath)


connection established, loading frame paths
relative paths of framelabels loaded


In [4]:
losses

[4.781259742018353,
 4.665066908827812,
 4.348992626941436,
 4.3357433239262555,
 2.3299958425363796,
 1.8817797854782174,
 1.9123016312016767,
 1.7096565538969988,
 1.5829626252103501,
 1.5223844651402891,
 1.4617407450115556,
 1.6932874113928265,
 1.5142941507949357,
 1.4429484103487524,
 1.437953687874358,
 1.3374671831125635,
 1.226515802275668]

In [5]:
pd.DataFrame(accuracies).transpose()

""


In [6]:
accuracies

{}

In [7]:
pd.DataFrame(final_classification_reports['Skill']).transpose()

,precision,recall,f1-score,support
jump,0.989501,0.974160,0.981771,387.000000
return from power,0.877778,0.918605,0.897727,86.000000
pushup,0.900000,0.947368,0.923077,76.000000
frog,0.927273,0.910714,0.918919,56.000000
split,0.846154,0.916667,0.880000,12.000000
SPAGAAT,0.000000,0.000000,0.000000,0.000000
crab,0.777778,0.875000,0.823529,8.000000
swift,0.000000,0.000000,0.000000,0.000000
mountainclimber,0.000000,0.000000,0.000000,0.000000
rad,0.000000,0.000000,0.000000,1.000000


In [8]:
pd.DataFrame(final_classification_reports['Turner1']).transpose()

,precision,recall,f1-score,support
normal,0.945614,0.981785,0.963360,549.000000
crouger,0.942857,0.846154,0.891892,39.000000
cross,0.850000,0.693878,0.764045,49.000000
cross BW,0.000000,0.000000,0.000000,2.000000
jump over cross BW,0.000000,0.000000,0.000000,2.000000
EB,0.615385,0.727273,0.666667,11.000000
toad,0.363636,0.500000,0.421053,8.000000
toad BW,0.000000,0.000000,0.000000,1.000000
EB toad,0.000000,0.000000,0.000000,4.000000
TS,0.000000,0.000000,0.000000,1.000000


In [9]:
pd.DataFrame(final_classification_reports['Turner2']).transpose()

,precision,recall,f1-score,support
normal,0.936620,0.974359,0.955117,546.000000
crouger,0.756757,0.777778,0.767123,36.000000
cross,0.853659,0.729167,0.786517,48.000000
cross BW,0.000000,0.000000,0.000000,5.000000
jump over cross BW,0.000000,0.000000,0.000000,1.000000
EB,0.416667,0.357143,0.384615,14.000000
toad,0.272727,0.375000,0.315789,8.000000
toad BW,0.000000,0.000000,0.000000,2.000000
EB toad,0.000000,0.000000,0.000000,5.000000
TS,0.000000,0.000000,0.000000,0.000000


In [10]:
pd.DataFrame(final_classification_reports['Type']).transpose()

,precision,recall,f1-score,support
Double Dutch,0.963947,0.990253,0.976923,513.000000
Single Dutch,0.000000,0.000000,0.000000,4.000000
Irish Dutch,0.000000,0.000000,0.000000,0.000000
Chinese Wheel,0.948980,0.978947,0.963731,95.000000
Transition,0.958333,0.821429,0.884615,28.000000
Snapperlike,0.900000,0.620690,0.734694,29.000000
accuracy,0.959641,0.959641,0.959641,0.959641
macro avg,0.628543,0.568553,0.593327,669.000000
weighted avg,0.953051,0.959641,0.954845,669.000000


In [11]:
pd.DataFrame(final_classification_reports['Hands']).transpose()

,precision,recall,f1-score,support
0,0.984946,0.944330,0.964211,485.00000
1,0.386364,0.395349,0.390805,43.00000
2,0.806250,0.914894,0.857143,141.00000
accuracy,0.902840,0.902840,0.902840,0.90284
macro avg,0.725853,0.751524,0.737386,669.00000
weighted avg,0.908810,0.902840,0.904789,669.00000


In [12]:
pd.DataFrame(final_classification_reports['Feet']).transpose()

,precision,recall,f1-score,support
0,0.976744,0.617647,0.756757,68.00000
1,0.465116,0.500000,0.481928,80.00000
2,0.924074,0.957774,0.940622,521.00000
accuracy,0.868460,0.868460,0.868460,0.86846
macro avg,0.788645,0.691807,0.726436,669.00000
weighted avg,0.874545,0.868460,0.867082,669.00000


In [13]:
pd.DataFrame(final_classification_reports['Rotations']).transpose()

,precision,recall,f1-score,support
0,0.000000,0.000000,0.000000,22.000000
1,0.917832,0.940860,0.929204,558.000000
2,0.493976,0.640625,0.557823,64.000000
3,0.461538,0.428571,0.444444,14.000000
4,1.000000,0.111111,0.200000,9.000000
5,0.000000,0.000000,0.000000,1.000000
8,0.000000,0.000000,0.000000,1.000000
accuracy,0.856502,0.856502,0.856502,0.856502
macro avg,0.410478,0.303024,0.304496,669.000000
weighted avg,0.835914,0.856502,0.840386,669.000000


In [14]:
pd.DataFrame(final_classification_reports['Turntable']).transpose()

,precision,recall,f1-score,support
0,0.981873,0.993884,0.987842,654.000000
1,0.285714,0.153846,0.200000,13.000000
2,0.000000,0.000000,0.000000,2.000000
accuracy,0.974589,0.974589,0.974589,0.974589
macro avg,0.422529,0.382577,0.395947,669.000000
weighted avg,0.965410,0.974589,0.969579,669.000000


In [15]:
pd.DataFrame(final_classification_reports['BodyRotations']).transpose()

,precision,recall,f1-score,support
0,0.997010,1.000000,0.998503,667.00000
1,0.000000,0.000000,0.000000,1.00000
2,0.000000,0.000000,0.000000,1.00000
accuracy,0.997010,0.997010,0.997010,0.99701
macro avg,0.332337,0.333333,0.332834,669.00000
weighted avg,0.994030,0.997010,0.995518,669.00000


In [16]:
pd.DataFrame(final_classification_reports['Backwards']).transpose()

,precision,recall,f1-score,support
0,0.990964,0.998483,0.994709,659.000000
1,0.800000,0.400000,0.533333,10.000000
accuracy,0.989537,0.989537,0.989537,0.989537
macro avg,0.895482,0.699241,0.764021,669.000000
weighted avg,0.988109,0.989537,0.987812,669.000000


In [17]:
pd.DataFrame(final_classification_reports['Sloppy']).transpose()

,precision,recall,f1-score,support
0,0.971599,1.000000,0.985595,650.000000
1,0.000000,0.000000,0.000000,19.000000
accuracy,0.971599,0.971599,0.971599,0.971599
macro avg,0.485800,0.500000,0.492798,669.000000
weighted avg,0.944005,0.971599,0.957604,669.000000


In [18]:
pd.DataFrame(final_classification_reports['Hard2see']).transpose()

,precision,recall,f1-score,support
0,0.992526,1.000000,0.996249,664.000000
1,0.000000,0.000000,0.000000,5.000000
accuracy,0.992526,0.992526,0.992526,0.992526
macro avg,0.496263,0.500000,0.498125,669.000000
weighted avg,0.985108,0.992526,0.988803,669.000000


In [19]:
pd.DataFrame(final_classification_reports['Fault']).transpose()

,precision,recall,f1-score,support
0,0.950673,1.000000,0.974713,636.000000
1,0.000000,0.000000,0.000000,33.000000
accuracy,0.950673,0.950673,0.950673,0.950673
macro avg,0.475336,0.500000,0.487356,669.000000
weighted avg,0.903778,0.950673,0.926633,669.000000
